In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models

In [2]:
# Define paths
train_dir = r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\train"
val_dir = r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\validate"

In [3]:
# Image parameters
img_width, img_height = 128, 128
batch_size = 32
epochs = 10

In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load data with flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 978 images belonging to 2 classes.
Found 246 images belonging to 2 classes.


In [5]:
# Calculate class weights to address imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights))

In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),  # Adjust input shape to match spectrogram size
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),  # Reduced size of dense layer to avoid overfitting
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Adjust for number of classes (2 classes for binary classification)
])

c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
import tensorflow as tf

# Define the corrected paths using raw string literals
train_dir = r'C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\train'
val_dir = r'C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\validate'

# Create the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True
)

# Create the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(128, 128)
)

Found 978 files belonging to 2 classes.
Found 246 files belonging to 2 classes.


In [11]:
# Compile the model with an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set up learning rate scheduler and early stopping callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with reduced epochs
history = model.fit(
    train_dataset,
    epochs=20,  # Start with 20 epochs
    validation_data=val_dataset,
    callbacks=[lr_schedule, early_stopping]
)

# Save the model
model.save('cnn_als_classifier.keras')

Epoch 1/20


31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 545ms/step - accuracy: 0.5610 - loss: 5.7964 - val_accuracy: 0.3821 - val_loss: 0.8503 - learning_rate: 0.0010
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 513ms/step - accuracy: 0.6658 - loss: 0.6581 - val_accuracy: 0.6179 - val_loss: 0.6726 - learning_rate: 9.5000e-04
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 536ms/step - accuracy: 0.6756 - loss: 0.6332 - val_accuracy: 0.6382 - val_loss: 0.6739 - learning_rate: 9.0250e-04
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 543ms/step - accuracy: 0.6522 - loss: 0.6233 - val_accuracy: 0.6626 - val_loss: 0.6704 - learning_rate: 8.5737e-04
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 18s 572ms/step - accuracy: 0.6620 - loss: 0.6022 - val_accuracy: 0.6667 - val_loss: 0.6770 - learning_rate: 8.1451e-04
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 553ms/step - accuracy: 0.6662 - loss: 0.6097 - val_accuracy: 0.6423 - val_loss: 0.7005 - learning_rate: 7.7378e-04
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 537ms/step - accuracy: 0.6490 -

In [13]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.6662 - loss: 0.6752
Validation Loss: 0.6703668236732483
Validation Accuracy: 0.6626016497612
